In [1]:
#packages needing installations across deliverables
!pip install CFEDemands
!pip install xarray
!pip install wbdata
!pip install -r requirements.txt

You are using pip version 9.0.3, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
#all imports neeeded for dataframing and visualizations
import datascience as ds
import numpy as np
import plotly.offline as py
import plotly.graph_objs as go
import pandas as pd
import wbdata as wb
from ipywidgets import interact, interactive, Dropdown, IntSlider
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.cm as cm
import cfe
import xarray
import fooddatacentral as fdc

In [3]:
#Using WBData to construct the GDP per capita in LCU (Local currency units) to eliminate the inflation of GDP per capita in USD
nigeria_country_code = 'NGA'
variable_labels = {'SP.POP.TOTL': 'Population'}
nigeria_pop = wb.get_dataframe(variable_labels, "NGA").loc[:, 'Population'].values[1]
nigeria_gdp = 'NY.GDP.MKTP.KN' 
years = [2010, 2012, 2015, 2018]
gdp_label = {nigeria_gdp :"GDP"}
nigeria_gdp = wb.get_dataframe(gdp_label, country = "NGA").loc[:, "GDP"].values[1]
nigeria_gdp_lcu = nigeria_gdp/nigeria_pop
print("Nigeria's GDP per capita in LCU is " + str(round(nigeria_gdp_lcu,2)))

Nigeria's GDP per capita in LCU is 360160.74


In [4]:
#taking this GDP per capita in LCU to determine the average weekly income
def nigeria_naira_converter_per_week(x):
    one_naira = 0.0028
    naira_to_usd = x*0.0028
    weekly_income = naira_to_usd/52
    return str(round(weekly_income,2))
print("Nigeria's per capita weekly income based on GDP per capita is $" + nigeria_naira_converter_per_week(nigeria_gdp_lcu))


Nigeria's per capita weekly income based on GDP per capita is $19.39


In [5]:
#function to pull data from Google Sheets and construct data frames that are input into CFE results
HH_and_Expenditure_Sheet = {'P3 HH/E Characteristics':'1RpMK9oo6knSG_eJMTdUJltq937WlIqAK0IE5inTLvfQ',
        }
Sheets = HH_and_Expenditure_Sheet
def dataframe_from_gspreadsheet(sheet_name, key):
    url = 'https://docs.google.com/spreadsheets/d/{key}/gviz/tq?tqx=out:csv&sheet={sheet_name}&headers=1'.format(
        key=key, sheet_name=sheet_name.replace(' ', '%20'))
    df = pd.read_csv(url)
    return df.drop([col for col in df.columns if col.startswith('Unnamed')], axis=1)
#Pulling the prices data from a Google Sheet
df = dataframe_from_gspreadsheet('Product Prices', HH_and_Expenditure_Sheet['P3 HH/E Characteristics'])
df['Price'] = df['Price'].astype(float)
df['Quantity'] = df['Quantity'].astype(float)
df['FDC_1'] = df['FDC_1'].astype(int)
df.head()

,Food,Quantity,Units,Price,FDC_1
0,Agricultural eggs,12.0,unit,0.11,748967
1,Apples,1.0,kg,2.68,559734
2,Bananas,2.0,kg,1.36,173944
3,Beef,4.0,kg,1.32,782828
4,Beer (local and imported),10.0,l,1.56,789593


In [7]:
#Importing FDC data for US nutrient data and using my personal API Key
import fooddatacentral as fdc
import warnings

user = "tyler"
apikey = {'tyler':"khbVdWdc5wfLYOMDftgM7jfTpihpDdtEuGmzud98"}
D = {}
count = 0
for food in  df.Food.tolist():
    try:
        FDC = df.loc[df.Food==food,:].FDC_1[count]
        count+=1
        D[food] = fdc.nutrients(apikey[user],FDC).Quantity
    except AttributeError: 
        warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC_1))        

D = pd.DataFrame(D,dtype=float)


In [8]:
# Convert food quantities to FDC units and drop any foods with missing data
df['FDC Quantity'] = df[['Quantity','Units']].T.apply(lambda x : fdc.units(x['Quantity'],x['Units']))
df.dropna(how='any')
D

,Agricultural eggs,Apples,Bananas,Beef,Beer (local and imported),Biscuits,Bottled water,Bread,Brown Beans,Cheese (wara),...,Local eggs,Okra,Onions,Orange/tangerine,Pineapple,Potatoes,Rice-local,"Soft drinks (Coca cola, spirit etc)",Tomatoes,Yams
10:0,NaN,NaN,0.001,0.000,0.000,NaN,0.00,NaN,NaN,NaN,...,NaN,NaN,0.000,0.000,NaN,0.001,NaN,0.00,0.000,NaN
12:0,NaN,NaN,0.002,0.003,0.000,NaN,0.00,NaN,NaN,NaN,...,NaN,NaN,0.000,0.000,NaN,0.004,NaN,0.00,0.000,NaN
14:0,0.032,NaN,0.002,0.108,0.000,NaN,0.00,NaN,NaN,NaN,...,0.032,NaN,0.004,0.001,NaN,0.001,NaN,0.00,0.000,NaN
14:1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15:0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15:1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16:0,2.320,NaN,0.102,1.091,0.000,NaN,0.00,NaN,NaN,NaN,...,2.320,NaN,0.034,0.021,NaN,0.021,NaN,0.00,0.020,NaN
16:1,NaN,NaN,0.010,0.154,0.000,NaN,0.00,NaN,NaN,NaN,...,NaN,NaN,0.000,0.005,NaN,0.001,NaN,0.00,0.001,NaN
16:1 c,0.238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17:0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Creating the diet requirements for seven individuals that made up the average Nigerian home
group_1 = "M 19-30"
group_2 = 'F 19-30'
group_3 = 'F 9-13'
bmin_1 = pd.read_csv('./diet_minimums.csv').set_index('Nutrition')[group_1]
bmin_2 = pd.read_csv('./diet_minimums.csv').set_index('Nutrition')[group_2]
bmin_3 = pd.read_csv('./diet_minimums.csv').set_index('Nutrition')[group_3]
bmin_10001 = (bmin_1 * 3) + (bmin_2 *3) + (bmin_3)
bmax_1 = pd.read_csv('./diet_maximums.csv').set_index('Nutrition')[group_1]
bmax_2 = pd.read_csv('./diet_maximums.csv').set_index('Nutrition')[group_2]
bmax_3 = pd.read_csv('./diet_maximums.csv').set_index('Nutrition')[group_3]
bmax_10001 = (bmax_1 * 3) + (bmax_2 *3) + (bmax_3)
bmin_10001

Nutrition
Energy                            14800.0
Protein                             340.0
Fiber, total dietary                207.2
Folate, DFE                        2700.0
Calcium, Ca                        7300.0
Carbohydrate, by difference         910.0
Iron, Fe                             86.0
Magnesium, Mg                      2370.0
Niacin                              102.0
Phosphorus, P                      5450.0
Potassium, K                      32700.0
Riboflavin                            8.1
Thiamin                               7.8
Vitamin A, RAE                     5400.0
Vitamin B-12                         16.2
Vitamin B-6                           8.8
Vitamin C, total ascorbic acid      540.0
Vitamin E (alpha-tocopherol)        101.0
Vitamin K (phylloquinone)           690.0
Zinc, Zn                             65.0
dtype: float64

In [12]:
#combining the list with prices and nutrients
df['FDC Quantity'] = df[['Quantity','Units']].T.apply(lambda x : fdc.units(x['Quantity'],x['Units']))
df['FDC Price'] = df['Price']/df['FDC Quantity']
df.dropna(how='any')
Prices = df.groupby('Food')['FDC Price'].min()

c = Prices.apply(lambda x:x.magnitude).dropna()
use = list(set(c.index.tolist()).intersection(D.columns.tolist()))
c = c[use]
c

Food
Brown Beans                            0.0110
Bottled water                          0.0036
Cheese (wara)                          0.1650
Tomatoes                               0.1340
Onions                                 0.1020
Apples                                 0.2680
Yams                                   0.0400
Beef                                   0.0330
Pineapple                              0.3600
Potatoes                               0.0720
Soft drinks (Coca cola, spirit etc)    0.0250
Fish                                   0.0225
Goat                                   0.3000
Bread                                  0.0316
Fresh milk                             0.0550
Rice-local                             0.0420
Biscuits                               0.0790
Chicken                                0.0440
Beer (local and imported)              0.0156
Okra                                   0.1800
Orange/tangerine                       0.3160
Bananas                      

In [13]:
# Drop nutritional information for foods with unknown prices or nutrients, unconstrained rows
Aall = D[c.index].fillna(0)
Amin = Aall.loc[bmin_10001.index]
Amax = Aall.loc[bmax_10001.index]
A = pd.concat([Amin,-Amax])
A

,Brown Beans,Bottled water,Cheese (wara),Tomatoes,Onions,Apples,Yams,Beef,Pineapple,Potatoes,...,Goat,Bread,Fresh milk,Rice-local,Biscuits,Chicken,Beer (local and imported),Okra,Orange/tangerine,Bananas
Nutrition,,,,,,,,,,,,,,,,,,,,,
Energy,0.00,0.00,187.00,18.000,166.000,46.00,118.00,146.000,51.00,397.000,...,142.000,231.000,62.00,386.00,347.00,298.000,43.000,44.00,405.000,371.000
Protein,25.60,0.00,6.67,0.880,1.100,0.00,1.33,18.500,0.00,2.630,...,26.870,11.540,3.33,7.02,8.00,25.730,0.460,3.00,1.500,1.090
"Fiber, total dietary",4.10,0.00,4.40,1.200,1.700,0.00,4.00,0.700,0.00,2.300,...,0.000,3.800,0.00,1.80,1.30,0.000,0.000,1.00,10.600,2.600
"Folate, DFE",0.00,0.00,0.00,15.000,19.000,0.00,0.00,16.000,0.00,26.000,...,5.000,0.000,0.00,0.00,0.00,6.000,6.000,0.00,30.000,20.000
"Calcium, Ca",158.00,3.00,22.00,10.000,23.000,0.00,20.00,20.000,0.00,18.000,...,17.000,212.000,83.00,70.00,400.00,21.000,4.000,80.00,161.000,5.000
"Carbohydrate, by difference",0.00,0.00,37.78,3.890,9.340,11.67,28.00,4.060,12.11,21.440,...,0.000,46.150,5.00,71.93,33.33,0.000,3.550,8.00,25.000,22.840
"Iron, Fe",4.70,0.00,2.40,0.270,0.210,0.00,0.48,1.880,0.00,1.070,...,3.700,3.850,0.00,1.26,2.40,1.410,0.020,0.54,0.800,0.260
"Magnesium, Mg",158.00,1.00,0.00,11.000,10.000,0.00,0.00,23.000,0.00,30.000,...,0.000,0.000,0.00,0.00,0.00,20.000,6.000,0.00,22.000,27.000
Niacin,0.00,0.00,0.00,0.594,0.116,0.00,0.00,4.402,0.00,1.348,...,3.917,5.385,0.00,0.00,0.00,6.662,0.513,0.00,0.900,0.665


In [14]:
#concatting the constraints
b = pd.concat([bmin_10001,-bmax_10001]) 
b

Nutrition
Energy                            14800.0
Protein                             340.0
Fiber, total dietary                207.2
Folate, DFE                        2700.0
Calcium, Ca                        7300.0
Carbohydrate, by difference         910.0
Iron, Fe                             86.0
Magnesium, Mg                      2370.0
Niacin                              102.0
Phosphorus, P                      5450.0
Potassium, K                      32700.0
Riboflavin                            8.1
Thiamin                               7.8
Vitamin A, RAE                     5400.0
Vitamin B-12                         16.2
Vitamin B-6                           8.8
Vitamin C, total ascorbic acid      540.0
Vitamin E (alpha-tocopherol)        101.0
Vitamin K (phylloquinone)           690.0
Zinc, Zn                             65.0
Sodium, Na                       -16000.0
dtype: float64

In [15]:
#importing package to solve the nutrient problem
from  scipy.optimize import linprog as lp
tol = 0
result = lp(c, -A, -b, method='interior-point')
result

     con: array([], dtype=float64)
     fun: 22.55923876015503
 message: 'Optimization terminated successfully.'
     nit: 16
   slack: array([6.64568179e-05, 1.36892245e+03, 1.68930629e+02, 4.99684131e+02,
       1.44870663e-03, 2.07141905e+02, 1.82040896e+02, 5.56392869e+03,
       1.78325579e+02, 2.29122991e+04, 7.21631278e+04, 4.41338971e+00,
       9.27984122e-01, 8.75475855e+02, 3.62769496e+01, 1.90306014e+01,
       1.70346528e+03, 3.05520160e-08, 7.46005846e+02, 2.65181870e+02,
       2.42779643e-05])
  status: 0
 success: True
       x: array([3.11304551e+01, 1.59780304e-06, 3.55019686e-08, 1.47254637e+02,
       9.18139814e-08, 2.73664481e-08, 2.69612991e-07, 4.13204327e+01,
       2.02347779e-08, 8.10699750e-07, 3.31786198e-07, 7.95950055e-08,
       2.24473715e-08, 1.64252005e-07, 1.29794394e-07, 2.30977462e-08,
       5.02239708e-08, 1.28976837e-07, 6.28608581e-07, 4.11635563e-08,
       3.72132961e-08, 1.64868810e+01])

In [16]:
#prints a series with the amounts of each food consumed and the total cost for meeting all the nutrients
diet = pd.Series(result.x,index=c.index)
print("\nYou'll be eating (in 100s of grams or milliliters) for a weekly cost of $" +str(round(result.fun*7,2)) + ":")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.


You'll be eating (in 100s of grams or milliliters) for a weekly cost of $157.91:
Food
Brown Beans                            3.113046e+01
Bottled water                          1.597803e-06
Cheese (wara)                          3.550197e-08
Tomatoes                               1.472546e+02
Onions                                 9.181398e-08
Apples                                 2.736645e-08
Yams                                   2.696130e-07
Beef                                   4.132043e+01
Pineapple                              2.023478e-08
Potatoes                               8.106998e-07
Soft drinks (Coca cola, spirit etc)    3.317862e-07
Fish                                   7.959501e-08
Goat                                   2.244737e-08
Bread                                  1.642520e-07
Fresh milk                             1.297944e-07
Rice-local                             2.309775e-08
Biscuits                               5.022397e-08
Chicken                      

In [17]:
#returns a Pandas dataframe with the recommended and consumed nutrients
family_nutrients = pd.DataFrame({"Outcome":np.abs(A).dot(diet),"Recommendation":np.abs(b)})
print("\n You'll be eating for a weekly cost of for a weekly cost of $" +str(round(result.fun*7,2)) + ":")
family_nutrients


 You'll be eating for a weekly cost of for a weekly cost of $157.91:


,Outcome,Recommendation
Nutrition,,
Energy,14800.000066,14800.0
Protein,1708.922448,340.0
"Fiber, total dietary",376.130629,207.2
"Folate, DFE",3199.684131,2700.0
"Calcium, Ca",7300.001449,7300.0
"Carbohydrate, by difference",1117.141905,910.0
"Iron, Fe",268.040896,86.0
"Magnesium, Mg",7933.928689,2370.0
Niacin,280.325579,102.0
